In [2]:
import pandas as pd

# === EXTRAÇÃO DOS DADOS ===
# Lê o arquivo CSV e armazena os dados em um DataFrame
df = pd.read_csv("dados_analise_cartao.csv")

# Extrai a coluna 'id' e converte os valores em uma lista Python
user_ids = df['id'].tolist()

# Exibe os IDs extraídos no console (apenas para conferência)
print(user_ids)

[1, 2, 3]


In [3]:
import json

# === FUNÇÃO DE CONSULTA DE USUÁRIO ===
def get_user(user_id):

  #Filtra o DataFrame para encontrar o usuário com o ID informado
  user_data = df[df['id'] == user_id]

  # Verifica se encontrou algum registro
  if not user_data.empty:
       # Converte a primeira linha encontrada em dicionário
    return user_data.iloc[0].to_dict()

   # Retorna None caso o usuário não exista
  return None

# === TRANSFORMAÇÃO DOS DADOS ===
# Cria uma lista de usuários válidos a partir da lista de IDs
users = [user for id in user_ids
         if (user := get_user(id)) is not None]

# === SAÍDA DOS DADOS ===
# Exibe os dados dos usuários em formato JSON formatado
print(json.dumps(users, indent=2))

[
  {
    "id": 1,
    "nome": "Ana",
    "sobrenome": "Silva",
    "numero": 11987654321,
    "email": "ana.silva@email.com",
    "numero_cartao": "4539 1488 0343 6467",
    "limite_cartao": 5000
  },
  {
    "id": 2,
    "nome": "Bruno",
    "sobrenome": "Souza",
    "numero": 21999887766,
    "email": "bruno.souza@email.com",
    "numero_cartao": "4716 4800 1234 5678",
    "limite_cartao": 3500
  },
  {
    "id": 3,
    "nome": "Carlos",
    "sobrenome": "Oliveira",
    "numero": 31988776655,
    "email": "carlos.oliveira@email.com",
    "numero_cartao": "4000 1234 5678 9010",
    "limite_cartao": 4200
  }
]


In [17]:
#!pip install google-genai
from google import genai
import os
import time


# === CONFIGURAÇÃO DA API ===
# Inicializa o cliente da API Gemini com a chave de acesso
client = genai.Client(
    api_key=os.getenv("GEMINI_API_KEY")
)

# === EXTRAÇÃO DOS DADOS ===
# Carrega o arquivo CSV contendo os nomes dos usuários
df = pd.read_csv("dados_analise_cartao.csv")

# === FUNÇÃO DE GERAÇÃO DE MENSAGEM ===
def gerar_mensagem(nome):
    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=(
                f"Você é um especialista em marketing bancário. "
                f"Crie uma mensagem curta e amigável para {nome} "
                f"explicando a importância dos investimentos. "
                f"Máximo de 100 caracteres."
            )
        )

         # Retorna apenas o texto gerado pelo modelo
        return response.text  # ✅ forma correta

    except Exception as e:
        # Retorna erro tratado para evitar quebra do fluxo
        return f"Erro para {nome}: {str(e)}"


# === PROCESSAMENTO DOS DADOS ===
# Gera mensagens personalizadas para cada nome do DataFrame
mensagens = []

for nome in df["nome"]:
    mensagem = gerar_mensagem(nome)
    mensagens.append(mensagem)
    print(f"{nome}: {mensagem}")

    # Pausa entre requisições para evitar limite de cota da API
    time.sleep(1)


# === CARGA DOS DADOS ===
# Adiciona a coluna de mensagens geradas ao DataFrame
df["mensagem"] = mensagens

# Salva o DataFrame atualizado em um novo arquivo CSV
# Esse arquivo será utilizado posteriormente para load e análises
df.to_csv("dados_analise_cartao_mensagem.csv", index=False, encoding="utf-8-sig")


Ana: Erro para Ana: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 34.104174806s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-flash', 'location': 'gl

In [18]:
# === LOAD DOS DADOS PROCESSADOS ===
# Carrega o arquivo CSV que contém as mensagens já geradas
df = pd.read_csv("dados_analise_cartao_mensagem.csv")

# Padroniza os nomes das colunas (remove espaços e converte para minúsculas)
df.columns = df.columns.str.strip().str.lower()

print("Load realizado com sucesso")

# Exibe no console o nome do usuário e a mensagem associada
for nome, mensagem in zip(df["nome"], df["mensagem"]):
    print(f"{nome}: {mensagem}\n")

Load realizado com sucesso
Ana: Erro para Ana: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 34.104174806s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-